# 🦾 Servo Gripper Control with Object Detection

**Workflow:**
1. ตรวจจับวัตถุด้วย YOLO
2. ผู้ใช้ยืนยันว่าจะหยิบวัตถุไหน
3. Gripper กางออกสุด
4. Robot เคลื่อนไปตำแหน่งวัตถุ แล้วลดลง Z=-63
5. Gripper หุบ (ตามความหนาวัตถุ)
6. Robot ยกขึ้น แล้วย้ายไปวาง

## 1️⃣ Imports & Setup

In [ ]:
!pip install pyserial pydobot matplotlib opencv-python numpy ultralytics

In [ ]:
import cv2
import numpy as np
import time
import socket
import serial
import importlib

import config
importlib.reload(config)

from ultralytics import YOLO

print("✓ Imports loaded")

## 2️⃣ Configuration

In [ ]:
# Robot IP
ROBOT_IP = '192.168.1.6'

# ESP32 Serial Port
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200

# Camera ID
CAMERA_ID = 2

# Homography Matrix (from calibrate.py)
HOMOGRAPHY_MATRIX = np.array([
    [-0.004699341952697413, -0.3137634530917012, 84.66801170689713],
    [-0.4056710916298707, -0.013981190139846252, 174.27196033222742],
    [-2.645420242327231e-06, -0.00024026660997854653, 1.0],
], dtype=np.float32)

# Z Heights (Online mode values)
Z_SAFE = -50      # ความสูงปลอดภัย
Z_GRASP = -64     # ความสูงหยิบวัตถุ
R_ANGLE = 0       # มุมหมุน Gripper

# Drop position
DROP_POS = (37.84, 100.97, -60, 0)  # X, Y, Z, R

# Servo angles
SERVO_OPEN = 22
SERVO_CLOSE = 96

print("✓ Configuration loaded")
print(f"  Robot IP: {ROBOT_IP}")
print(f"  ESP32 Port: {ESP32_PORT}")
print(f"  Camera ID: {CAMERA_ID}")

## 3️⃣ GripperController Class (ESP32 Serial)

In [ ]:
class GripperController:
    """ควบคุม Servo Gripper ผ่าน ESP32 Serial"""
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.current_angle = SERVO_OPEN
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)  # Wait for ESP32 to reset
            
            # Clear buffer
            self.serial.reset_input_buffer()
            
            print(f"✅ Gripper connected on {self.port}")
            return True
        except Exception as e:
            print(f"❌ Gripper connection failed: {e}")
            return False
    
    def disconnect(self):
        if self.serial:
            self.serial.close()
            print("Gripper disconnected")
    
    def send_command(self, cmd):
        """ส่งคำสั่งไป ESP32 และรอ response"""
        if not self.serial:
            print("❌ Gripper not connected")
            return None
        
        self.serial.write((cmd + '\n').encode())
        time.sleep(0.1)
        
        response = ""
        while self.serial.in_waiting:
            response += self.serial.readline().decode().strip() + "\n"
        
        return response.strip()
    
    def open(self):
        """กางแขนออกสุด (22°)"""
        print("🦾 Opening gripper...")
        response = self.send_command('O')
        self.current_angle = SERVO_OPEN
        print(f"   Response: {response}")
        return response
    
    def close(self):
        """หุบแขนเข้าสุด (96°)"""
        print("🦾 Closing gripper...")
        response = self.send_command('C')
        self.current_angle = SERVO_CLOSE
        print(f"   Response: {response}")
        return response
    
    def grip(self, angle):
        """หุบไปที่มุมที่กำหนด"""
        angle = max(SERVO_OPEN, min(SERVO_CLOSE, int(angle)))
        print(f"🦾 Gripping to angle {angle}°...")
        response = self.send_command(f'G{angle}')
        self.current_angle = angle
        print(f"   Response: {response}")
        return response
    
    def get_status(self):
        """รับสถานะปัจจุบัน"""
        return self.send_command('?')
    
    def calculate_grip_angle(self, object_width_pixels, pixels_per_mm=3.0):
        """
        คำนวณมุม Servo จากความกว้างวัตถุ
        
        Args:
            object_width_pixels: ความกว้างวัตถุในหน่วย pixels
            pixels_per_mm: อัตราส่วน pixel ต่อ mm (ต้อง calibrate)
        
        Returns:
            angle: มุม Servo (22-96)
        """
        object_width_mm = object_width_pixels / pixels_per_mm
        
        # Linear mapping:
        # 22° (open) = gripper at widest (~74° range = ~74mm est.)
        # 96° (close) = gripper at narrowest (~0mm)
        # Simplified: angle ≈ 96 - (width_mm * (74/74)) = 96 - width_mm
        
        OPEN_ANGLE = SERVO_OPEN
        CLOSE_ANGLE = SERVO_CLOSE
        OPEN_WIDTH_MM = 74  # Gripper ที่มุม 22° กว้าง ~74mm
        
        # Calculate angle based on width
        angle = CLOSE_ANGLE - (object_width_mm * (CLOSE_ANGLE - OPEN_ANGLE) / OPEN_WIDTH_MM)
        
        # Clamp + safety margin
        grip_angle = max(OPEN_ANGLE, min(CLOSE_ANGLE, angle + 5))  # +5° for tight grip
        
        print(f"   Object: {object_width_pixels}px = {object_width_mm:.1f}mm → angle: {grip_angle:.0f}°")
        return int(grip_angle)

print("✓ GripperController class loaded")

## 4️⃣ DobotControllerTCP Class

In [ ]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock_dashboard = None
        self.homography_matrix = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock_dashboard = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock_dashboard.settimeout(5)
            self.sock_dashboard.connect((ip, self.dashboard_port))
            
            print("Clearing Errors...")
            self.send_command("ClearError()")
            time.sleep(0.5)
            
            print("Enabling Robot...")
            self.send_command("EnableRobot()")
            time.sleep(4)
            
            # Use User(1), Tool(1) to match Online mode
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            
            print("\n✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Connection Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock_dashboard:
            try:
                self.sock_dashboard.send((cmd + "\n").encode("utf-8"))
                return self.sock_dashboard.recv(1024).decode("utf-8")
            except Exception as e:
                print(f"Command Error: {e}")
        return None

    def home(self):
        print("🤖 Moving to HOME...")
        self.send_command("JointMovJ(0.19,-12.21,38.29,-0.34)")
        time.sleep(4)
        print("✅ HOME")

    def move_to(self, x, y, z, r=0):
        """เคลื่อนที่ไปยังตำแหน่ง"""
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   Sending: {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait_time=3):
        """เคลื่อนที่และรอ"""
        self.move_to(x, y, z, r)
        time.sleep(wait_time)

    def pixel_to_robot(self, u, v):
        """แปลง pixel → robot coordinates"""
        if self.homography_matrix is None:
            return None, None
        point = np.array([u, v, 1], dtype=np.float32)
        result = np.dot(self.homography_matrix, point)
        return result[0] / result[2], result[1] / result[2]

print("✓ DobotControllerTCP class loaded")

## 5️⃣ Connect Devices

In [ ]:
# Connect Gripper (ESP32)
gripper = GripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
gripper.connect()

# Get status
status = gripper.get_status()
print(f"Gripper status: {status}")

In [ ]:
# Connect Robot (Dobot MG400)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)

if robot.connect(ROBOT_IP):
    print("\n" + "="*50)
    print(f"🤖 Robot connected to {ROBOT_IP}!")
    print("="*50)
else:
    print("⚠️ Failed to connect to robot")

In [ ]:
# Load YOLO model
yolo = YOLO('yolov8n.pt')
print("✓ YOLO model loaded")

## 6️⃣ 🦾 Manual Gripper Test

In [ ]:
# Test Open
gripper.open()

In [ ]:
# Test Close
gripper.close()

In [ ]:
# Test Grip to specific angle
gripper.grip(60)  # ลองหุบไปที่ 60°

## 7️⃣ 🎯 Pick-and-Place Workflow

**ขั้นตอน:**
1. กล้องตรวจจับวัตถุ → แสดง bounding box
2. คลิกเลือกวัตถุที่ต้องการหยิบ
3. กด `SPACE` เพื่อยืนยัน
4. Robot + Gripper ทำงานอัตโนมัติ:
   - Gripper เปิด
   - Robot ไปตำแหน่งวัตถุ (Z=-50)
   - Robot ลดลง (Z=-63)
   - Gripper หุบ
   - Robot ยกขึ้น (Z=-30)
   - Robot ย้ายไปวาง
   - Gripper เปิด

In [ ]:
# Move robot to home
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

In [ ]:
# =================================================================
# 🎯 Pick-and-Place with Object Detection
# =================================================================

selected_object = None  # Object to pick
detected_objects = []   # All detected objects

def mouse_callback_pick(event, x, y, flags, param):
    global selected_object
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Find object at clicked position
        for obj in detected_objects:
            x1, y1, x2, y2, conf, cls_name, width = obj
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_object = obj
                cx = (x1 + x2) // 2
                cy = (y1 + y2) // 2
                robot_x, robot_y = robot.pixel_to_robot(cx, cy)
                print(f"\n📦 Selected: {cls_name}")
                print(f"   Pixel: ({cx}, {cy})")
                print(f"   Robot: ({robot_x:.1f}, {robot_y:.1f})")
                print(f"   Width: {width}px")
                print(f"\n   Press SPACE to pick, R to reset, Q to quit")
                break

def pick_object(obj):
    """Execute pick-and-place sequence"""
    x1, y1, x2, y2, conf, cls_name, width = obj
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    
    print("\n" + "="*50)
    print(f"🤖 Picking: {cls_name}")
    print("="*50)
    
    # Step 1: Open gripper
    print("\n[1/7] Opening gripper...")
    gripper.open()
    time.sleep(5)
    
    # Step 2: Move to position (safe height)
    print(f"\n[2/7] Moving to position ({robot_x:.1f}, {robot_y:.1f}, {Z_SAFE})...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=3)
    
    # Step 3: Lower to grasp height
    print(f"\n[3/7] Lowering to Z={Z_GRASP}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_GRASP, R_ANGLE, wait_time=2)
    
    # Step 4: Calculate grip angle and close
    print(f"\n[4/7] Calculating grip angle...")
    grip_angle = gripper.calculate_grip_angle(width)
    gripper.grip(grip_angle)
    time.sleep(1)
    
    # Step 5: Lift up
    print(f"\n[5/7] Lifting up to Z={Z_SAFE}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=2)
    
    # Step 6: Move to drop position
    drop_x, drop_y, drop_z, drop_r = DROP_POS
    print(f"\n[6/7] Moving to drop position ({drop_x}, {drop_y}, {drop_z})...")
    robot.move_to_and_wait(drop_x, drop_y, drop_z, drop_r, wait_time=3)
    
    # Step 7: Release
    print("\n[7/7] Releasing object...")
    gripper.open()
    time.sleep(1)
    
    print("\n" + "="*50)
    print("✅ Pick-and-Place Complete!")
    print("="*50)

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("="*50)
    print("🎯 Pick-and-Place Mode")
    print("="*50)
    print("Click on object to select → SPACE to pick → Q to quit")
    print("="*50)
    
    cv2.namedWindow('Pick and Place')
    cv2.setMouseCallback('Pick and Place', mouse_callback_pick)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect objects
        results = yolo(frame, verbose=False)
        detected_objects = []
        
        for r in results:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls_id = int(box.cls[0])
                cls_name = yolo.names[cls_id]
                width = x2 - x1
                
                detected_objects.append((x1, y1, x2, y2, conf, cls_name, width))
                
                # Draw bounding box
                color = (0, 255, 0)  # Green
                if selected_object and (x1, y1) == (selected_object[0], selected_object[1]):
                    color = (0, 0, 255)  # Red for selected
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{cls_name} {conf:.2f}", (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Draw instructions
        cv2.putText(frame, "Click to select | SPACE=Pick | Q=Quit", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        if selected_object:
            cv2.putText(frame, f"Selected: {selected_object[5]}", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        cv2.imshow('Pick and Place', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord('r'):
            selected_object = None
            print("\n🔄 Selection reset")
        elif key == ord(' ') and selected_object:
            pick_object(selected_object)
            selected_object = None
    
    cap.release()
    cv2.destroyAllWindows()
    print("\nCamera closed")

## 8️⃣ 🔧 Manual Pick (No YOLO)

In [ ]:
# Move robot to home
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

In [ ]:
# =================================================================
# Manual Pick - คลิกบนภาพแล้ว Robot ไปหยิบ
# =================================================================

click_pos = None

def mouse_callback_manual(event, x, y, flags, param):
    global click_pos
    if event == cv2.EVENT_LBUTTONDOWN:
        click_pos = (x, y)
        robot_x, robot_y = robot.pixel_to_robot(x, y)
        print(f"\n📍 Clicked: Pixel({x}, {y}) → Robot({robot_x:.1f}, {robot_y:.1f})")
        print("   Press SPACE to pick at this location")

def manual_pick(px, py, grip_angle=80):
    """Manual pick at pixel position with specified grip angle"""
    robot_x, robot_y = robot.pixel_to_robot(px, py)
    
    print("\n" + "="*50)
    print(f"🤖 Manual Pick at ({robot_x:.1f}, {robot_y:.1f})")
    print("="*50)
    
    # Step 1: Open
    print("[1] Opening gripper...")
    gripper.open()
    time.sleep(4)
    
    # Step 2: Move to safe height
    print(f"[2] Moving to position (Z={Z_SAFE})...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=3)
    time.sleep(1)
    
    # Step 3: Lower
    print(f"[3] Lowering to Z={Z_GRASP}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_GRASP, R_ANGLE, wait_time=2)
    time.sleep(1)

    # Step 4: Grip
    print(f"[4] Gripping at {grip_angle}°...")
    gripper.grip(grip_angle)
    time.sleep(4)
    
    # Step 5: Lift
    print(f"[5] Lifting to Z={Z_SAFE}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=2)
    time.sleep(1)
    
    # Step 6: Move to drop
    drop_x, drop_y, drop_z, drop_r = DROP_POS
    print(f"[6] Moving to drop position...")
    robot.move_to_and_wait(drop_x, drop_y, drop_z, drop_r, wait_time=3)
    time.sleep(1)
    
    # Step 7: Release
    print("[7] Releasing...")
    gripper.open()
    time.sleep(1)
    
    print("\n✅ Manual Pick Complete!")

# Camera loop
cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("="*50)
    print("🔧 Manual Pick Mode")
    print("="*50)
    print("Click anywhere → SPACE to pick → Q to quit")
    print("="*50)
    
    cv2.namedWindow('Manual Pick')
    cv2.setMouseCallback('Manual Pick', mouse_callback_manual)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Draw click position
        if click_pos:
            cv2.circle(frame, click_pos, 10, (0, 0, 255), -1)
            cv2.circle(frame, click_pos, 15, (0, 0, 255), 2)
        
        cv2.putText(frame, "Click to mark | SPACE=Pick | Q=Quit", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Manual Pick', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord(' ') and click_pos:
            grip_angle = 80  # Default grip angle - adjust as needed
            manual_pick(click_pos[0], click_pos[1], grip_angle)
            click_pos = None
    
    cap.release()
    cv2.destroyAllWindows()

## 9️⃣ Home & Cleanup

In [ ]:
# Move robot to home
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

In [ ]:
# Disconnect gripper
gripper.disconnect()